In [55]:
import pandas as pd

from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  roc_auc_score

In [2]:
data = pd.read_csv('./mcdonalds.csv')

In [4]:
data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,policies_violated,policies_violated:confidence,city,policies_violated_gold,review,Unnamed: 10
0,679455653,False,finalized,3,2/21/15 0:36,RudeService\nOrderProblem\nFilthy,1.0\n0.6667\n0.6667,Atlanta,NaN,"I'm not a huge mcds lover, but I've been to be...",NaN
1,679455654,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,Terrible customer service. ŒæI came in at 9:30...,NaN
2,679455655,False,finalized,3,2/21/15 0:26,SlowService\nOrderProblem,1.0\n1.0,Atlanta,NaN,"First they ""lost"" my order, actually they gave...",NaN
3,679455656,False,finalized,3,2/21/15 0:27,na,0.6667,Atlanta,NaN,I see I'm not the only one giving 1 star. Only...,NaN
4,679455657,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,"Well, it's McDonald's, so you know what the fo...",NaN


In [5]:
data.shape

(1525, 11)

In [7]:
data = data[pd.notnull(data['policies_violated'])]

In [8]:
data.shape

(1471, 11)

In [10]:
data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,policies_violated,policies_violated:confidence,city,policies_violated_gold,review,Unnamed: 10
0,679455653,False,finalized,3,2/21/15 0:36,RudeService\nOrderProblem\nFilthy,1.0\n0.6667\n0.6667,Atlanta,NaN,"I'm not a huge mcds lover, but I've been to be...",NaN
1,679455654,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,Terrible customer service. ŒæI came in at 9:30...,NaN
2,679455655,False,finalized,3,2/21/15 0:26,SlowService\nOrderProblem,1.0\n1.0,Atlanta,NaN,"First they ""lost"" my order, actually they gave...",NaN
3,679455656,False,finalized,3,2/21/15 0:27,na,0.6667,Atlanta,NaN,I see I'm not the only one giving 1 star. Only...,NaN
4,679455657,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,"Well, it's McDonald's, so you know what the fo...",NaN


In [28]:
data["rude"] = data["policies_violated"].str.contains("RudeService").apply(lambda i: int(i))

/home/climbus/anaconda3/lib/python3.4/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [26]:
data

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,policies_violated,policies_violated:confidence,city,policies_violated_gold,review,Unnamed: 10,rude
0,679455653,False,finalized,3,2/21/15 0:36,RudeService\nOrderProblem\nFilthy,1.0\n0.6667\n0.6667,Atlanta,NaN,"I'm not a huge mcds lover, but I've been to be...",NaN,1
1,679455654,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,Terrible customer service. ŒæI came in at 9:30...,NaN,1
2,679455655,False,finalized,3,2/21/15 0:26,SlowService\nOrderProblem,1.0\n1.0,Atlanta,NaN,"First they ""lost"" my order, actually they gave...",NaN,0
3,679455656,False,finalized,3,2/21/15 0:27,na,0.6667,Atlanta,NaN,I see I'm not the only one giving 1 star. Only...,NaN,0
4,679455657,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,"Well, it's McDonald's, so you know what the fo...",NaN,1
5,679455658,False,finalized,3,2/21/15 0:13,BadFood\nSlowService,0.7111\n0.6444,Atlanta,NaN,This has to be one of the worst and slowest Mc...,NaN,0
6,679455659,False,finalized,3,2/21/15 0:36,SlowService\nScaryMcDs,0.6562\n0.6562,Atlanta,NaN,I'm not crazy about this McDonald's. ŒæThis is...,NaN,0
7,679455660,False,finalized,3,2/21/15 0:15,RudeService,0.6801,Atlanta,NaN,One Star and I'm beng kind. I blame management...,NaN,1
8,679455661,False,finalized,3,2/21/15 0:29,SlowService\nRudeService\nMissingFood,1.0\n1.0\n0.6667,Atlanta,NaN,Never been upset about any fast food drive thr...,NaN,1
9,679455662,False,finalized,3,2/21/15 0:12,na,1,Atlanta,NaN,This McDonald's has gotten much better. Usuall...,NaN,0


In [29]:
X = data['review']
y = data['rude']

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [38]:
cv = CountVectorizer()

In [39]:
X_train_dtm = cv.fit_transform(X_train)

In [40]:
X_test_dtm = cv.transform(X_test)

In [51]:
nb = MultinomialNB()

In [52]:
nb.fit(X_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [53]:
pred_proba = nb.predict_proba(X_test_dtm)[:, 1]

In [54]:
roc_auc_score(y_test, pred_proba)

0.84196471149260832

In [56]:
logreg = LogisticRegression()

In [57]:
logreg.fit(X_train_dtm, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [58]:
pred_proba_lr = logreg.predict_proba(X_test_dtm)[:, 1]

In [59]:
roc_auc_score(y_test, pred_proba_lr)

0.8233985058019393